# Market/Property Evaluation Tool for Residential Real Estate Investors

Our project is designed to provide residential real estate investors with a tool that uses calculated metrics to help narrow their search for investment properties based on a set of criteria.  These metrics can be both market (city) and property based so that investors can start by narrowing from a list of cities, then filter individual properties within that market to determine which rental houses show attractive investment ratios.

**Questions our tool will answer**
1. *"In which US market should I buy a rental property?"*
2. *"Which individual properties with in a market should I take a closer look at?"*

## Core Metrics

### We will use a set of core metrics to evaluate and filter data on a market and property level. A list of these metrics can be found below, along with their calculation and goal outcome.

**Market Level**

1. Price to Rent Ratio: Median Home Price / Annual Median Rent, *Goal: <15*

2. Annual Population Growth % (last 5 years)

3. Annual Job Growth % (last 5 years)

4. Quality of Life Index


**Property Level**

1. Rent Ratio: Monthly Rent Estimate / Total Cost, *Goal: >1%*

2. Cap Rate:  Net Operating Income (NOI) / Total Cost, *Goal: >7%*

3. OpEx Ratio:  Operating Expenses/Gross Income, *Goal: <70%*

4. Price vs Comparables:  (List Price - Comp Price) / Price *Goal: >10%*


## Equations and Variables

### Other Formulas

1.  Total Cost = List Price + Closing Fees
2.  NOI = Gross Income - Annual Operating Expenses (OpEx)
3.  OpEx = Repairs/Maintenance + Annual Property Tax + Vacancy Rate + Insurance Estimate + Misc Expenses

### Based on the above calculations, we will need to gather the following individual data points (variable and potential source listed below):

1.  Address/List Price (US Real Estate API)
2.  Closing Fees (Formula: List Price * 3%)
3.  Gross Rent (US Real Estate API: Monthly Rent Estimate * 12)
4.  Annual Property Tax (US Real Estate API)
5.  Insurance Estimate (Formula: (List Price/100) * $3.50
6.  Vacancy Rate (Formula: Gross Rent * 7%)
7.  Repairs/Maintenance (Formula: Gross Rent * 10%)
8.  Misc Expenses (Formula: Gross Rent * 5%)
9.  Median Home Price (Zillow csv: "Zillow Home Value Index")
10. Annual Median Rent (Zillow csv: "Zillow Observed Rent Index")
11. City Housing Supply (Zillow API)
12. City Housing Demand (Zillow API)
13. Quality of Life Index (Numbeo)

## Output

The results for each individual property would show the following:

Address
List Price
Total Cost
Gross Rent
Rent Ratio
Cap Rate
OpEx Ratio
Price vs Comp






In [21]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import json

In [17]:
load_dotenv()
api_key = os.getenv("X_RAPID_API_KEY")

In [163]:

#Pull in US Real Estate API
url = "https://us-real-estate.p.rapidapi.com/for-sale"

querystring = {"offset":"200","limit":"1000","state_code":"TX","city":"Houston","sort":"newest"}

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "us-real-estate.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

#Display API Data
data = response.json()
usre_data_hou = json.dumps(data, indent=4)

#Convert Data to a Dictionary
new_dict = json.loads(usre_data_hou)
usre_data_hou_dict = new_dict["data"]["results"]



In [167]:
new_list = []
for prop in usre_data_hou_dict:
    new_dict = {
        "property_id": prop["property_id"],
        "list_price": prop["list_price"],
        "postal_code": prop["location"]["address"]["postal_code"],
        "city": prop["location"]["address"]["city"],
        "line": prop["location"]["address"]["line"],
        "state_code": prop["location"]["address"]["state_code"],
    }
    new_list.append(new_dict)

df = pd.DataFrame(new_list)

{'postal_code': '77005',
 'state': 'Texas',
 'coordinate': {'lon': -95.427906, 'lat': 29.710792},
 'city': 'West University Place',
 'state_code': 'TX',
 'line': '3211 Pittsburgh St'}

In [161]:
#Convert Dictionary to a DataFrame and Clean Data
hou_df = pd.DataFrame.from_dict(usre_data_hou_dict, orient='columns')
hou_df_new = hou_df.iloc[:, 10:22]
hou_df_parsed = hou_df_new.drop(columns=["lead_attributes", "photos", "flags", "community", "products", "virtual_tours", "other_listings", "listing_id", "price_reduced_amount"])
hou_df_indexed = hou_df_parsed.set_index("property_id")
hou_df_indexed.head()


,list_price,location
property_id,,
8678257509,2799000,"{'address': {'postal_code': '77005', 'state': ..."
8552525367,450000,"{'address': {'postal_code': '77018', 'state': ..."
8717560148,279990,"{'address': {'postal_code': '77063', 'state': ..."
8926070952,3595000,"{'address': {'postal_code': '77024', 'state': ..."
8229191530,1259000,"{'address': {'postal_code': '77005', 'state': ..."


In [63]:
print(json.dumps(data, indent=4))

{
    "status": 200,
    "data": {
        "total": 8768,
        "count": 200,
        "results": [
            {
                "primary_photo": {
                    "href": "https://ap.rdcpix.com/bddbfda409a15e1afeea9f6b1c985dedl-m1606122086s-w1024_h768.jpg"
                },
                "last_update_date": "2023-02-14T15:17:59Z",
                "source": {
                    "plan_id": null,
                    "agents": [
                        {
                            "office_name": null
                        }
                    ],
                    "spec_id": null,
                    "type": "mls"
                },
                "tags": null,
                "permalink": "8289-Park-Place-Blvd_Houston_TX_77017_M73934-76875",
                "status": "for_sale",
                "list_date": "2023-02-13T19:40:10Z",
                "open_houses": null,
                "description": {
                    "baths_3qtr": null,
                    "year_built":